# Peer Review Assignment - Data Engineer - ETL


## Objectives

In this final part you will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment. Others may need to be installed by you. The cells below will install these libraries when executed.


In [7]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [58]:
import glob
import json
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section


In [59]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2022-09-10 19:53:07--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-09-10 19:53:09 (2.62 GB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2022-09-10 19:53:10--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [23]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

In [ ]:
# Load the file 'exchange_rates.csv'. 
df = pd.read_csv('exchange_rates.csv')
# Find the exchange rate for Great British Pounds with the symbol GBP, and enter the numeric value.
exchange_rate = df[df['Currency'] == 'GBP']['Exchange Rate'].values[0]


## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [24]:
def extract():
    extracted_data = pd.DataFrame(columns=['Name','Market Cap (US$ Billion)'])
    # Extract data from JSON files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    return extracted_data

<b>Question 1</b> Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [32]:
# Write your code here
df = pd.read_csv('exchange_rates.csv', index_col=0)
exchange_rate = df.loc['GBP']['Rates']
exchange_rate

0.7323984208000001

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [46]:
def transform(exchange_rate, extracted_data):
    # Find the exchange rate of USD to GBP
    df = pd.read_csv('exchange_rates.csv', index_col=0)
    usd_to_gbp = df.loc['USD', 'Rates'] / exchange_rate
    # Change the column Market Cap (US$ Billion) type to float
    extracted_data['Market Cap (US$ Billion)'] = extracted_data['Market Cap (US$ Billion)'].astype(float)
    # Changes the Market Cap (US$ Billion) column from USD to GBP
    extracted_data['Market Cap (US$ Billion)'] = extracted_data['Market Cap (US$ Billion)'] * usd_to_gbp

    # Rounds the Market Cap (US$ Billion) column to 3 decimal places
    extracted_data['Market Cap (US$ Billion)'] = extracted_data['Market Cap (US$ Billion)'].round(3)

    # Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)
    extracted_data.rename(columns={'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=True)

    transformed_data = extracted_data

    return transformed_data

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [29]:
def load(df_data):
    df_data.to_csv('bank_market_cap_gbp.csv', index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [28]:
def log(message):
    timestamp_format = '%H:%M:%S-%d/%m/%Y'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("ETL_Assignment.log", "a") as log_file:
        log_file.write(timestamp + ", " + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [60]:
# Write your code here
log("ETL Job Started")
# Extract function
log("Extract Phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [61]:
# Call the function here
extracted_data = extract()
# Print first 5 rows
extracted_data.head(5)

/var/folders/dk/_cfdg6bd67zbn7r4nmk7qpnm0000gn/T/ipykernel_1831/2687455119.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
/var/folders/dk/_cfdg6bd67zbn7r4nmk7qpnm0000gn/T/ipykernel_1831/2687455119.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)


,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [62]:
# Write your code here
log("Extract Phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [63]:
# Write your code here
log("Transform Phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [64]:
# Call the function here
transformed_data = transform(exchange_rate,extracted_data)
# Print the first 5 rows here
i = 0
while i <= 5:
    print(transformed_data.iloc[i])
    i += 1 

Name                         JPMorgan Chase
Market Cap (GBP$ Billion)           533.772
Name: 0, dtype: object
Name                         Industrial and Commercial Bank of China
Market Cap (GBP$ Billion)                                    471.347
Name: 1, dtype: object
Name                         Bank of America
Market Cap (GBP$ Billion)            444.199
Name: 2, dtype: object
Name                         Wells Fargo
Market Cap (GBP$ Billion)        420.554
Name: 3, dtype: object
Name                         China Construction Bank
Market Cap (GBP$ Billion)                    351.447
Name: 4, dtype: object
Name                         HSBC Holdings PLC
Market Cap (GBP$ Billion)              299.386
Name: 5, dtype: object


Log your data <code>"Transform phase Ended"</code>


In [65]:
# Write your code here
log("Transforma Phase Ended")

### Load


Log the following `"Load phase Started"`.


In [66]:
# Write your code here
log("Load Phase Started")

Call the load function


In [67]:
# Write your code here
load(transformed_data)

Log the following `"Load phase Ended"`.


In [68]:
# Write your code here
log("Load Phase Ended")
log("ETL Job Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
